In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as opt
import quantecon as qe

## Question 2: Solve for representative agent steady state

In [3]:
def params1():
    
    θ = 0.21
    ν = 0.64
    δ = 0.1
    β = 0.96
    ϕ = 0.5
    ρ = 0.9
    σ = 0.02
    cf = 0.01
    ce = 0.1
    
    params = θ, ν, δ, β, ϕ, ρ, σ, cf, ce
    
    return params

In [56]:
def get_markov_shocks(*params):
    
    # unpack params
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    # markov approximation of shocks using quantecon
    shock_matrix = qe.markov.approximation.tauchen(ρ, σ, n = 10).state_values

    return shock_matrix

def get_SS_shocks(markov_shocks):
    
    SS_shock = markov_shocks.stationary_distributions
    
    return SS_shock

def get_r(K, N, shock, *params):
    
    # unpack params
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    r = θ * np.exp(shock) * K**(θ-1) * N**ν
    
    return r

def get_w(K, N, shock, *params):
    
    # unpack params
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    w = ν * np.exp(shock) * K**θ * N**(ν-1)
    
    return w

def get_c(w, r, K, N, *params):
    
    # unpack params
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    c = w * N + r * K - (K - (1 - δ) * K)
    
    return c

def get_pi(w, r, K, N, shock, *params):
    
    # unpack params
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    pi = np.exp(shock) * K**θ * n**ν - w*N
    
    return pi

def SS_rep(K, N, shock, *params):
    
    # unpack params
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    w = get_w(K, N, shock, *params)
    
    f1 = -1 - ϕ*δ + β*(θ * K**(θ-1) * N**ν - (ϕ/2) * δ**2 + (ϕ - 1)*δ + 1)
    
    return f1

def model_SS(K_guess, N, shock, *params):
    
    K_ss = opt.fsolve(SS_rep, K_guess, args = (N, shock, *params))
    w_ss = get_w(K_ss, N, shock, *params)
    r_ss = get_r(K_ss, N, shock, *params)
                      
    return K_ss,w_ss,r_ss

model_SS(1, 0.6, 0, *params1())

(array([1.04511682]), array([0.77637249]), array([0.14625]))

## Question 3: Solve for incumben firms decision rule
- follow similar Euler equation for Q2, adding stochastic shocks,  entry and exit.

In [101]:
def V1_fun(k, kp, ϵ, w_rep, V_guess, Markov_chain, *params):
    
    # unpack params
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    # labour supply
    n = (w_rep/(ν*np.exp(ϵ)*(k**θ)))**(1/(ν-1))

    EV = np.dot(V_guess, Markov_chain.T)

    V1 = np.exp(ϵ)*(k**(θ))*(n**(ν)) - w_rep*n - (kp - (1-δ)*k) - k*(ϕ/2)*(kp/k - (1-δ))**2 + β * EV

    return V1

def iteration(Markov_chain, w_rep, N, tol, maxiter, *params, show = True):
    
    # unpack params:
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    # set capital grid:
    klow = 0.01
    khigh = 1.5
    k_grid = np.linspace(klow, khigh, N)
    
    # set value grid and the equilibrium value gird:
    v_grid = 0.01 * np.ones((N, 10))
    v_eqbm = np.zeros_like(v_grid)
    
    # set policy grid:
    policy = np.zeros_like(v_grid)
    
    # iterate over value function:
    for iters in range(maxiter):
        if show == True:
            print('iteration {}'.format(iters))
        for i in range(N): # first loop through all k's
            k = k_grid[i]
            for j in range(10): # then loop through all shocks
                ϵ = Markov_chain[j]
                max_value = -np.inf
                index = 0
                for m in range(N): # loop through all k's again to find best next period k
                    kp = k_grid[m]
                    V_guess = v_grid[m, :]
                    
                    potential_max = V1_fun(k, kp, ϵ, w_rep, V_guess, Markov_chain, *params)

                    if (1-δ)*k > potential_max - cf: # exit condition
                        if max_value < (1-δ)*k:
                            max_value = (1-δ)*k
                            index = -khigh # if exit gives maximum value, 

                    else:
                        if max_value < potential_max - cf:
                            max_value = potential_max - cf
                            index = m
                
                v_eqbm[i, j] = max_value # update v_eqbm
                
                # track policy function
                if index >= 0:
                    policy[i, j] = k_grid[index]
                else:
                    policy[i, j] = index
        diff = np.max(np.abs(v_eqbm - v_grid))
        if show == True:
            print('maximum value difference is {}'.format(diff))
        if diff < tol:
            break
        v_grid = v_eqbm.copy()
            
    return v_eqbm, policy

In [102]:
markov_chain = get_markov_shocks(*params1())
w_rep = float(model_SS(1, 0.6, 0, *params1())[2])

In [103]:
iteration(markov_chain, w_rep, 50, 1e-6, 1000, *params1(), show = True)

iteration 0
maximum value difference is 10.24186265127915
iteration 1
maximum value difference is 0.387728253562865
iteration 2
maximum value difference is 1.7763568394002505e-15


(array([[ 0.32482102,  0.34529136,  0.36757707,  0.39183915,  0.41825287,
          0.44700903,  0.47831538,  0.51239808,  0.54950333,  0.58989919],
        [ 0.6938714 ,  0.74009872,  0.79042563,  0.84521567,  0.90486467,
          0.96980352,  1.04050135,  1.11746889,  1.20126214,  1.29248644],
        [ 0.94769808,  1.01182439,  1.08163761,  1.15764209,  1.24038687,
          1.33046972,  1.4285414 ,  1.53531037,  1.65154795,  1.77809384],
        [ 1.15713326,  1.23611772,  1.32210677,  1.41572159,  1.51763847,
          1.62859364,  1.74938867,  1.88089617,  2.02406616,  2.17993291],
        [ 1.34158231,  1.43364575,  1.53387365,  1.64299008,  1.76178329,
          1.89111145,  2.03190883,  2.18519255,  2.35206995,  2.53374655],
        [ 1.50979626,  1.61372587,  1.72687228,  1.85005287,  1.98415749,
          2.13015492,  2.28909986,  2.46214052,  2.65052696,  2.8556201 ],
        [ 1.66611991,  1.78101704,  1.9061036 ,  2.04228322,  2.19053967,
          2.35194396,  2.5276620